In [7]:
#general tools
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import sqlite3

#data processing tools
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_squared_log_error
from sklearn.model_selection import cross_val_score
from scipy import stats
import shap
from sklearn.neighbors import KNeighborsRegressor

#model building tools
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
import lightgbm as lgb
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from catboost import Pool
import statsmodels.api as sm

#model productionalization tools
import pickle


ModuleNotFoundError: No module named 'shap'

### Import database

In [9]:
#Establish connection with the local trulia housing .db 
conn = sqlite3.connect(r'trulia_sb_house_data.db')

#read the "cleaned data" table from database and store in a pandas dataframe
df = pd.read_sql_query("SELECT * FROM trulia_house_SB_data_cleaned", conn)


In [10]:

df.head()
df.columns


Index(['level_0', 'index', 'price', 'address', 'zip', 'num_bedrooms',
       'num_baths', 'building_sqft', 'year_built', 'lot_area',
       'home_description', 'feature_list', 'house_age', 'has_garage',
       'has_ocean_views', 'has_mountain_views', 'has_pool', 'has_upstairs',
       'has_IV', 'building_sqft_was_missing'],
      dtype='object')

In [12]:
#dropping unneeded columns
df_model=df.drop(['level_0','index', 'address', 'year_built', 'home_description', 'feature_list'],axis=1)

### Feature Engineering

In [13]:
#The only catagorical feature is the zipcode
enc = OneHotEncoder(handle_unknown='ignore')

enc_df = pd.DataFrame(enc.fit_transform(df[['zip']]).toarray())
enc_df.columns = enc.get_feature_names(['zip'])

df_model = df_model.join(enc_df)
df_model.head()


/Users/dylannguyen/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,price,zip,num_bedrooms,num_baths,building_sqft,lot_area,house_age,has_garage,has_ocean_views,has_mountain_views,...,building_sqft_was_missing,zip_93067,zip_93101,zip_93103,zip_93105,zip_93108,zip_93109,zip_93110,zip_93111,zip_93117
0,3.900000,93108,3.0,3.0,2.354,1.00,62.0,0,0,0,...,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,1.780000,93105,3.0,2.0,1.395,0.14,73.0,1,0,0,...,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,1.350000,93105,3.0,2.0,1.344,0.50,50.0,0,0,0,...,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,2.899000,93109,3.0,3.0,2.558,0.50,42.0,0,0,0,...,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,1.824215,93109,4.0,2.0,1.407,0.14,63.0,0,0,0,...,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [15]:
#number of nulls in each column
print(df_model.isnull().sum())

price                        0
zip                          0
num_bedrooms                 0
num_baths                    0
building_sqft                0
lot_area                     0
house_age                    0
has_garage                   0
has_ocean_views              0
has_mountain_views           0
has_pool                     0
has_upstairs                 0
has_IV                       0
building_sqft_was_missing    0
zip_93067                    0
zip_93101                    0
zip_93103                    0
zip_93105                    0
zip_93108                    0
zip_93109                    0
zip_93110                    0
zip_93111                    0
zip_93117                    0
dtype: int64


### Imputation

In [14]:
#imputation: missing data are filled in to create a complete data matrix that can be analyzed using standard methods
#takes in df and imputes missing numeric values with a knn regressor
def knn_imputer(df):

    #isolate the columns with categorical data
    cat_data = df.select_dtypes(exclude=[np.number])

    #isolate the columns with numeric data
    numeric_data = df.select_dtypes(include=[np.number])

    nan_columns = numeric_data.columns[numeric_data.isna().any()].tolist()
    no_nan_columns = numeric_data.drop(columns=nan_columns).columns

    for col in nan_columns:

        KNN_test_data = numeric_data[numeric_data[col].isna()]
        KNN_train_data = numeric_data.dropna()
        KNN_model = KNeighborsRegressor(n_neighbors=3)
        KNN_model.fit(KNN_train_data[no_nan_columns], KNN_train_data[col])
        numeric_data.loc[numeric_data[col].isna(), col] = KNN_model.predict(
            KNN_test_data[no_nan_columns])

    return pd.concat([numeric_data, cat_data], axis=1)


#df_model=knn_imputer(df_model)
df_model.dropna(subset=['building_sqft'], inplace=True)


In [16]:
#printing to make sure no columns have null values
print(df_model.isnull().sum())


price                        0
zip                          0
num_bedrooms                 0
num_baths                    0
building_sqft                0
lot_area                     0
house_age                    0
has_garage                   0
has_ocean_views              0
has_mountain_views           0
has_pool                     0
has_upstairs                 0
has_IV                       0
building_sqft_was_missing    0
zip_93067                    0
zip_93101                    0
zip_93103                    0
zip_93105                    0
zip_93108                    0
zip_93109                    0
zip_93110                    0
zip_93111                    0
zip_93117                    0
dtype: int64


In [ ]:
#Finalized descriptive statistics for each numeric column in df
df_model.describe()